### Imports

In [1]:
import pandas as pd
import numpy as np

#Python Standard Libs Imports
import json
import urllib2
import sys
from datetime import datetime
from os.path import isfile, join, splitext
from glob import glob
import os
import time
#from distributed import Executor, hdfs

### Functions

#### Basic Functions

#### OTP Functions

#### Analysis Functions

In [2]:
def get_df_stats(df,filtered_df,df_label,filtered_df_label):
    df_size = df.count()
    filtered_df_size = filtered_df.count()
    print "Total", df_label,":", df_size
    print "Total", filtered_df_label, ":", filtered_df_size, "(", 100*(filtered_df_size/float(df_size)), "%)"

def get_filtered_df_stats(filtered_df,full_df_size,filtered_df_label,full_df_label):
    filtered_df_size = filtered_df.count()
    print filtered_df_label, "in Total", full_df_label, ":", filtered_df_size, "(", 100*(filtered_df_size/float(full_df_size)), "%)"

### Main Code

#### Reading Input Variables

In [3]:
#initial_date = datetime.strptime('2017-07-01', '%Y-%m-%d')
#final_date = datetime.strptime('2017-07-01', '%Y-%m-%d')
#od_matrix_folderpath = '/local/tarciso/masters/data/bus_trips/test/single-day-test/2017_07_01/od/trips_od/'
#buste_data_folderpath = '/local/tarciso/masters/data/bus_trips/test/single-day-test/2017_07_01/buste/'
#otp_server_url = 'http://150.165.85.4:10402/otp/'
#results_folderpath = '/local/tarciso/masters/data/bus_trips/test/single-day-test/2017_07_01'

In [4]:
initial_date = datetime.strptime('2017-05-08', '%Y-%m-%d')
final_date = datetime.strptime('2017-05-08', '%Y-%m-%d')
od_matrix_folderpath = '/local/tarciso/data/trips-optimality-exp/single-day-test/2017_05_08/od/trips_od/'
buste_data_folderpath = '/local/tarciso/data/trips-optimality-exp/single-day-test/2017_05_08/buste'
otp_server_url = 'http://150.165.85.4:10402/otp/'
results_folderpath = '/local/tarciso/data/trips-optimality-exp/single-day-test/2017_05_08/itineraries'

#### Reading OD Matrix

In [5]:
start_time = time.time()
print "Reading OD-Matrix Data..."
od_matrix_day_folderpath = od_matrix_folderpath + '/' + initial_date.strftime('%Y_%m_%d') + '_od'
od_matrix = pd.read_csv(od_matrix_day_folderpath + os.sep + 'part-00000')

Reading OD-Matrix Data...


In [6]:
od_matrix[['date']].head(4)

,date
0,1494201600
1,1494201600
2,1494201600
3,1494201600


In [7]:
print "Fixing OD Matrix dates due to bug on date saving on cluster..."
SECONDS_OFFSET = 10800
od_matrix.loc[:,'date'] = pd.to_datetime(od_matrix['date'], unit='s').dt.strftime('%Y-%m-%d')
od_matrix['user_trip_id'] = od_matrix['o_boarding_id']

Fixing OD Matrix dates due to bug on date saving on cluster...


In [8]:
od_matrix[['date']].head(4)

,date
0,2017-05-08
1,2017-05-08
2,2017-05-08
3,2017-05-08


In [9]:
od_matrix.dtypes

route                      int64
tripNum                    int64
shapeId                    int64
shapeSequence              int64
shapeLat                 float64
shapeLon                 float64
distanceTraveledShape    float64
busCode                   object
gpsPointId               float64
gpsLat                   float64
gpsLon                   float64
distanceToShapePoint     float64
timestamp                 object
stopPointId                int64
problem                   object
birthdate                 object
cardTimestamp             object
lineName                  object
gender                    object
date                      object
id                         int64
o_route                    int64
o_bus_code                object
o_date                     int64
o_tripNum                  int64
o_timestamp               object
o_shape_id                 int64
o_shape_seq                int64
o_shape_lat              float64
o_shape_lon              float64
o_stop_id 

In [10]:
print "Preprocessing Data..."

def advance_od_matrix_start_time(od_matrix,extra_seconds):
    return od_matrix.assign(o_datetime = lambda x: pd.to_datetime(x['date'] + ' ' + x['o_timestamp']), 
                            d_datetime = lambda x: pd.to_datetime(x['date'] + ' ' + x['timestamp'])) \
                    .assign(executed_duration = lambda x: (x['d_datetime'] - x['o_datetime']) / pd.Timedelta(minutes=1),
                            o_base_datetime = lambda x: pd.to_datetime(x['o_datetime']) - pd.Timedelta(minutes=2))
    
od_matrix = advance_od_matrix_start_time(od_matrix,120)


Preprocessing Data...


In [11]:
#raw_od_matrix = od_matrix.copy()

In [12]:
#od_matrix = raw_od_matrix.head(50)

In [13]:
#len(od_matrix)

50

#### Getting OTP suggested itineraries

In [14]:
def get_router_id(query_date):
    INTERMEDIATE_OTP_DATE = datetime.strptime("2017-06-30", "%Y-%m-%d")
    
    router_id = ''
    date_timestamp = datetime.strptime(query_date, "%Y-%m-%d")   
    
    if (date_timestamp <= INTERMEDIATE_OTP_DATE):
        return 'ctba-2017-1'
    else:
        return 'ctba-2017-2'

def get_otp_itineraries(otp_url,o_lat,o_lon,d_lat,d_lon,date,time,verbose=False):
    otp_http_request = 'routers/{}/plan?fromPlace={},{}&toPlace={},{}&mode=TRANSIT,WALK&date={}&time={}'
	
    router_id = get_router_id(date)
    otp_request_url = otp_url + otp_http_request.format(router_id,o_lat,o_lon,d_lat,d_lon,date,time)
    if verbose:
        print otp_request_url
    return json.loads(urllib2.urlopen(otp_request_url).read())

def get_otp_suggested_trips(od_matrix,otp_url):
    trips_otp_response = {}
    counter = 0
    for index, row in od_matrix.iterrows():
        id=long(row['user_trip_id'])
        start_time = row['o_timestamp']
        trip_plan = get_otp_itineraries(otp_url,row['o_shape_lat'], row['o_shape_lon'], row['shapeLat'], row['shapeLon'],row['date'],start_time)
        trips_otp_response[id] = trip_plan
        counter+=1

    return trips_otp_response

def get_otp_scheduled_trips(od_matrix,otp_url):
    trips_otp_response = {}
    counter = 0
    for index, row in od_matrix.iterrows():
        id=long(row['user_trip_id'])
        start_time = row['o_timestamp']
        trip_plan = get_executed_trip_schedule(otp_url,row['o_shape_lat'], row['o_shape_lon'], row['shapeLat'], row['shapeLon'],
                                               row['date'],start_time,row['route'],row['o_stop_id'])
        trips_otp_response[id] = trip_plan
        counter+=1

    return trips_otp_response

print "Getting OTP suggested itineraries..."
otp_suggestions = get_otp_suggested_trips(od_matrix,otp_server_url)

Getting OTP suggested itineraries...


In [73]:
#otp_suggestions

In [16]:
def extract_otp_trips_legs(otp_trips):
    trips_legs = []

    for trip in otp_trips.keys():
        if 'plan' in otp_trips[trip]:
            itinerary_id = 1
            for itinerary in otp_trips[trip]['plan']['itineraries']:
                date = otp_trips[trip]['plan']['date']/1000
                leg_id = 1
                for leg in itinerary['legs']:
                    route = leg['route'] if leg['route'] != '' else None
                    fromStopId = leg['from']['stopId'].split(':')[1] if leg['mode'] == 'BUS' else None
                    toStopId = leg['to']['stopId'].split(':')[1] if leg['mode'] == 'BUS' else None
                    start_time = long(leg['startTime'])/1000
                    end_time = long(leg['endTime'])/1000
                    duration = (end_time - start_time)/60
                    trips_legs.append((date,trip,itinerary_id,leg_id,start_time,end_time,leg['mode'],route,fromStopId,toStopId, duration))
                    leg_id += 1
                itinerary_id += 1
    return trips_legs

def prepare_otp_legs_df(otp_legs_list):
    labels=['date','user_trip_id','itinerary_id','leg_id','otp_start_time','otp_end_time','mode','route','from_stop_id','to_stop_id','otp_duration_mins']
    return pd.DataFrame.from_records(data=otp_legs_list, columns=labels) \
                    .assign(date = lambda x: pd.to_datetime(x['date'],unit='s').dt.strftime('%Y-%m-%d'),
                            otp_duration_mins = lambda x : (x['otp_end_time'] - x['otp_start_time'])/60,
                            route = lambda x : pd.to_numeric(x['route'],errors='coerce'),
                            from_stop_id = lambda x : pd.to_numeric(x['from_stop_id'],errors='coerce'),
                            to_stop_id = lambda x : pd.to_numeric(x['to_stop_id'],errors='coerce')) \
                    .assign(otp_start_time = lambda x : pd.to_datetime(x['otp_start_time'], unit='s'),
                            otp_end_time = lambda x : pd.to_datetime(x['otp_end_time'], unit='s')) \
                    .sort_values(by=['date','user_trip_id','itinerary_id','otp_start_time'])
    
print "Extracting OTP Legs info..."
otp_legs_df = prepare_otp_legs_df(extract_otp_trips_legs(otp_suggestions))

#otp_suggestions = None

Extracting OTP Legs info...


In [17]:
otp_legs_df.head(5)

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins
92,2017-05-08,17179869196,1,1,2017-05-08 11:07:58,2017-05-08 11:07:59,WALK,NaN,NaN,NaN,0.016667
93,2017-05-08,17179869196,1,2,2017-05-08 11:08:00,2017-05-08 11:31:20,BUS,901.0,29598.0,33654.0,23.333333
94,2017-05-08,17179869196,1,3,2017-05-08 11:31:21,2017-05-08 11:31:42,WALK,NaN,NaN,NaN,0.350000
95,2017-05-08,17179869196,2,1,2017-05-08 11:16:58,2017-05-08 11:16:59,WALK,NaN,NaN,NaN,0.016667
96,2017-05-08,17179869196,2,2,2017-05-08 11:17:00,2017-05-08 11:40:20,BUS,901.0,29598.0,33654.0,23.333333


In [18]:
otp_legs_df.dtypes

date                         object
user_trip_id                  int64
itinerary_id                  int64
leg_id                        int64
otp_start_time       datetime64[ns]
otp_end_time         datetime64[ns]
mode                         object
route                       float64
from_stop_id                float64
to_stop_id                  float64
otp_duration_mins           float64
dtype: object

#### Gathering schedule info for executed trips

In [19]:
def get_executed_trip_schedule(otp_url,o_lat,o_lon,d_lat,d_lon,date,time,route,start_stop_id,verbose=False):
    DEF_AGENCY_NAME = 'URBS'
    DEF_AGENCY_ID = 1
    otp_http_request = 'routers/{}/plan?fromPlace={},{}&toPlace={},{}&mode=TRANSIT,WALK&date={}&time={}&numItineraries=1&preferredRoutes={}_{}&startTransitStopId={}_{}&maxWalkingDistance=150&maxTransfers=0'
    
    router_id = get_router_id(date)
    otp_request_url = otp_url + otp_http_request.format(router_id,o_lat,o_lon,d_lat,d_lon,date,time,DEF_AGENCY_NAME,route,DEF_AGENCY_ID,start_stop_id)
    if verbose:
        print otp_request_url
    return json.loads(urllib2.urlopen(otp_request_url).read())

print "Getting OTP schedule info for executed trips..."
executed_trips_schedule = get_otp_scheduled_trips(od_matrix,otp_server_url)

Getting OTP schedule info for executed trips...


In [20]:
print "Extracting OTP Legs info..."
executed_trips_schedule_df = prepare_otp_legs_df(extract_otp_trips_legs(executed_trips_schedule)) \
                                [lambda x: x['mode'] == 'BUS'] \
                                .assign(itinerary_id = 0)

#executed_trips_schedule = None

Extracting OTP Legs info...


In [21]:
executed_trips_schedule_df.head()

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins
30,2017-05-08,17179869196,0,2,2017-05-08 11:08:00,2017-05-08 11:31:20,BUS,901.0,29598.0,33654.0,23.333333
77,2017-05-08,94489280525,0,2,2017-05-08 21:28:00,2017-05-08 21:50:13,BUS,972.0,29599.0,33375.0,22.216667
103,2017-05-08,163208757264,0,2,2017-05-08 11:33:43,2017-05-08 11:38:56,BUS,821.0,34147.0,33342.0,5.216667
93,2017-05-08,377957122060,0,2,2017-05-08 14:57:24,2017-05-08 15:09:51,BUS,924.0,26811.0,38141.0,12.450000
53,2017-05-08,455266533398,0,2,2017-05-08 08:52:23,2017-05-08 09:23:55,BUS,924.0,38142.0,27373.0,31.533333


In [22]:
executed_trips_schedule_df.sort_values(['user_trip_id']).head(5).dtypes

date                         object
user_trip_id                  int64
itinerary_id                  int64
leg_id                        int64
otp_start_time       datetime64[ns]
otp_end_time         datetime64[ns]
mode                         object
route                       float64
from_stop_id                float64
to_stop_id                  float64
otp_duration_mins           float64
dtype: object

In [23]:
od_matrix[['date','user_trip_id','o_base_datetime','route','o_stop_id','stopPointId','next_o_stop_id']].sort_values(['user_trip_id']).head(5)

,date,user_trip_id,o_base_datetime,route,o_stop_id,stopPointId,next_o_stop_id
28,2017-05-08,17179869196,2017-05-08 07:57:32,901,29598,33654,33653
44,2017-05-08,94489280525,2017-05-08 18:02:31,972,29599,33375,33308
33,2017-05-08,111669149696,2017-05-08 07:18:56,822,33801,33784,33801
13,2017-05-08,163208757264,2017-05-08 08:31:15,40,34147,33342,33348
35,2017-05-08,377957122060,2017-05-08 11:49:30,924,26758,38141,38141


In [24]:
od_matrix \
        .rename(index=str, columns={'o_stop_id':'from_stop_id', 'stopPointId':'to_stop_id'}) \
        [['date','user_trip_id','o_base_datetime','route','from_stop_id','to_stop_id','next_o_stop_id']] \
        .sort_values('user_trip_id').dtypes

date                       object
user_trip_id                int64
o_base_datetime    datetime64[ns]
route                       int64
from_stop_id                int64
to_stop_id                  int64
next_o_stop_id              int64
dtype: object

In [25]:
matched_executed_trips = od_matrix \
                            .rename(index=str, columns={'o_stop_id':'from_stop_id', 'stopPointId':'to_stop_id'}) \
                            .merge(executed_trips_schedule_df, 
                                 on=['date','user_trip_id','from_stop_id','to_stop_id'], how='inner') \
                            [['date','user_trip_id','itinerary_id','otp_duration_mins','otp_start_time']] \
                            .rename(index=str, columns={'otp_duration_mins':'planned_duration_mins', 'otp_start_time':'planned_start_time'})

In [26]:
matched_executed_trips.head()

,date,user_trip_id,itinerary_id,planned_duration_mins,planned_start_time
0,2017-05-08,1451698946054,0,15.983333,2017-05-08 13:31:16
1,2017-05-08,867583393796,0,5.150000,2017-05-08 13:30:00
2,2017-05-08,816043786247,0,14.366667,2017-05-08 09:11:36
3,2017-05-08,996432412675,0,7.083333,2017-05-08 10:41:53
4,2017-05-08,1477468749834,0,20.750000,2017-05-08 10:08:30


In [27]:
len(matched_executed_trips)

27

In [28]:
len(od_matrix)

50

In [29]:
len(executed_trips_schedule_df)

40

#### Printing Itineraries Stats

In [30]:
total_num_itineraries = len(otp_legs_df[['user_trip_id','itinerary_id']].drop_duplicates())
total_num_legs = len(otp_legs_df)
num_bus_legs = len(otp_legs_df[otp_legs_df['mode'] == 'BUS'])

print "Total num itineraries:", total_num_itineraries
print "Total num legs:", total_num_legs
print "Total num bus legs:", num_bus_legs, '(', 100*(num_bus_legs/float(total_num_legs)), '%)'


Total num itineraries: 148
Total num legs: 420
Total num bus legs: 138 ( 32.8571428571 %)


#### Matching OTP Bus Legs Origins with BUSTE Data

In [31]:
print "Reading BUSTE data..."
buste_day_folderpath = buste_data_folderpath + '/' + initial_date.strftime('%Y_%m_%d')
bus_trips_data = pd.read_csv(buste_day_folderpath + os.sep + 'part-00000') \
                    .assign(tripNum = lambda x: pd.to_numeric(x['tripNum'], errors='coerce'))

Reading BUSTE data...


/home/tarciso/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [32]:
bus_trips_data.head()

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,distanceToShapePoint,timestamp,stopPointId,problem,birthdate,cardTimestamp,lineName,cardNum,gender,date
0,901,1.0,2257,6191075,-25.402164,-49.327866,298.503,MC076,-,-,...,-,06:09:23,32369,BETWEEN,-,-,-,-,-,2017_05_08
1,901,1.0,2257,6191091,-25.403712,-49.328166,588.304,MC076,-,-,...,-,06:09:52,33651,BETWEEN,-,-,-,-,-,2017_05_08
2,901,1.0,2257,6191099,-25.404864,-49.327253,817.779,MC076,-,-25.404913,...,5.7547846,06:10:42,33653,NO_PROBLEM,-,-,-,-,-,2017_05_08
3,901,1.0,2257,6191127,-25.408142,-49.324268,1300.149,MC076,-,-,...,-,06:12:01,33657,BETWEEN,-,-,-,-,-,2017_05_08
4,901,1.0,2257,6191158,-25.411098,-49.320234,1880.640,MC076,-,-25.411081,...,6.2103405,06:12:42,33660,NO_PROBLEM,-,-,-,-,-,2017_05_08


In [33]:
bus_trips_data.dtypes

route                      int64
tripNum                  float64
shapeId                    int64
shapeSequence              int64
shapeLat                 float64
shapeLon                 float64
distanceTraveledShape    float64
busCode                   object
gpsPointId                object
gpsLat                    object
gpsLon                    object
distanceToShapePoint      object
timestamp                 object
stopPointId                int64
problem                   object
birthdate                 object
cardTimestamp             object
lineName                  object
cardNum                   object
gender                    object
date                      object
dtype: object

In [34]:
def clean_buste_data(buste_data):
    return buste_data[["date","route","busCode","tripNum","stopPointId","timestamp"]] \
                        .dropna(subset=["date","route","busCode","tripNum","stopPointId","timestamp"]) \
                        .drop_duplicates(subset=['date','route','busCode','tripNum','stopPointId']) \
                        .assign(route = lambda x: pd.to_numeric(x['route'], errors='coerce'),
                                timestamp = lambda x: pd.to_datetime(x['date'] + ' ' + x['timestamp'], format='%Y_%m_%d %H:%M:%S', errors='coerce')) \
                        .assign(date = lambda x: x['date'].str.replace('_','-'))

clean_bus_trips_data = clean_buste_data(bus_trips_data)

In [35]:
clean_bus_trips_data.dtypes

date                   object
route                   int64
busCode                object
tripNum               float64
stopPointId             int64
timestamp      datetime64[ns]
dtype: object

In [36]:
clean_bus_trips_data[['route','date','timestamp']].head()

,route,date,timestamp
0,901,2017-05-08,2017-05-08 06:09:23
1,901,2017-05-08,2017-05-08 06:09:52
2,901,2017-05-08,2017-05-08 06:10:42
3,901,2017-05-08,2017-05-08 06:12:01
4,901,2017-05-08,2017-05-08 06:12:42


In [37]:
def find_otp_bus_legs_actual_start_time(otp_legs_df,clean_bus_trips_df):
    legs_buste_match = otp_legs_df.assign(stopPointId = otp_legs_df['from_stop_id']) \
            .merge(clean_bus_trips_df, on=['date','route','stopPointId'], how='inner') \
            .dropna(subset=['timestamp']) \
            .assign(timediff = lambda x: np.absolute(x['timestamp'] - x['otp_start_time'])) \
            .drop('otp_duration_mins', axis=1)
    
    earliest_legs_start_times = legs_buste_match.groupby(['date','user_trip_id','itinerary_id','route','from_stop_id']) \
                                .timediff.min().reset_index()
        
    legs_st_time = legs_buste_match.merge(earliest_legs_start_times, on=['date','user_trip_id','itinerary_id','route','from_stop_id','timediff'], how='inner') \
                [['date','user_trip_id','itinerary_id','leg_id','route','busCode','tripNum','from_stop_id','otp_start_time','timestamp','to_stop_id','otp_end_time']] \
                .rename(index=str, columns={'timestamp':'from_timestamp'}) \
                .assign(route = lambda x: pd.to_numeric(x['route']))
    
    return legs_st_time
            
    
print "Finding OTP Bus Legs Actual Start Times in Bus Trips Data..."
otp_legs_st = find_otp_bus_legs_actual_start_time(otp_legs_df,clean_bus_trips_data)

Finding OTP Bus Legs Actual Start Times in Bus Trips Data...


In [38]:
otp_legs_st.head()

,date,user_trip_id,itinerary_id,leg_id,route,busCode,tripNum,from_stop_id,otp_start_time,from_timestamp,to_stop_id,otp_end_time
0,2017-05-08,17179869196,1,2,901.0,MC305,7.0,29598.0,2017-05-08 11:08:00,2017-05-08 11:12:05,33654.0,2017-05-08 11:31:20
1,2017-05-08,17179869196,2,2,901.0,MC305,7.0,29598.0,2017-05-08 11:17:00,2017-05-08 11:12:05,33654.0,2017-05-08 11:40:20
2,2017-05-08,17179869196,3,2,901.0,MC303,5.0,29598.0,2017-05-08 11:27:00,2017-05-08 11:24:52,33654.0,2017-05-08 11:51:16
3,2017-05-08,386547056643,2,2,901.0,MC302,5.0,29598.0,2017-05-08 10:39:00,2017-05-08 10:42:05,26720.0,2017-05-08 10:40:20
4,2017-05-08,755914244105,2,2,901.0,MC080,19.0,29598.0,2017-05-08 22:41:00,2017-05-08 22:36:34,33681.0,2017-05-08 22:50:33


#### Printing Matched OTP Bus Legs Origins Stats

In [39]:
num_bus_legs_st = len(otp_legs_st)
print "Num Bus Legs whose start was found:", num_bus_legs_st, '(', 100*(num_bus_legs_st/float(num_bus_legs)), '%)'


Num Bus Legs whose start was found: 134 ( 97.1014492754 %)


#### Cleaning Memory

In [40]:
#Clean memory
#otp_legs_df.unpersist(blocking=True)
#bus_trips_data.unpersist(blocking=True)
#clean_bus_trips_data.unpersist(blocking=True)

#### Matching OTP Bus Legs Destinations with BUSTE Data

In [41]:
def find_otp_bus_legs_actual_end_time(otp_legs_st,clean_bus_trips):
    legs_buste_match = otp_legs_st.assign(stopPointId = otp_legs_st['to_stop_id']) \
            .merge(clean_bus_trips_data, on=['date','route','busCode','tripNum','stopPointId'], how='inner') \
            .dropna(subset=['timestamp']) \
            .assign(timediff = lambda x: np.absolute(x['timestamp'] - x['otp_end_time'])) \
                    .rename(index=str,columns={'timestamp':'to_timestamp'}) \
                    .sort_values(by=['date','route','to_stop_id','timediff'])
    return legs_buste_match

print "Finding OTP Bus Legs Actual End Times in Bus Trips Data..."
otp_legs_start_end = find_otp_bus_legs_actual_end_time(otp_legs_st,clean_bus_trips_data)

Finding OTP Bus Legs Actual End Times in Bus Trips Data...


In [42]:
otp_legs_start_end.head()

,date,user_trip_id,itinerary_id,leg_id,route,busCode,tripNum,from_stop_id,otp_start_time,from_timestamp,to_stop_id,otp_end_time,stopPointId,to_timestamp,timediff
21,2017-05-08,163208757264,3,2,40,MB301,3.0,34147.0,2017-05-08 11:49:25,2017-05-08 12:17:47,33342.0,2017-05-08 11:54:55,33342,2017-05-08 12:24:04,00:29:09
20,2017-05-08,163208757264,2,2,40,MB301,3.0,34147.0,2017-05-08 11:35:16,2017-05-08 12:17:47,33342.0,2017-05-08 11:40:53,33342,2017-05-08 12:24:04,00:43:11
88,2017-05-08,936302870530,1,2,150,MN611,5.0,26796.0,2017-05-08 10:55:56,2017-05-08 10:56:44,32795.0,2017-05-08 11:07:37,32795,2017-05-08 11:04:58,00:02:39
56,2017-05-08,558345748483,3,1,226,BA123,15.0,34487.0,2017-05-08 15:57:57,2017-05-08 15:59:59,1899.0,2017-05-08 16:01:00,1899,2017-05-08 15:26:19,00:34:41
55,2017-05-08,558345748483,2,1,226,BA112,14.0,34487.0,2017-05-08 15:38:57,2017-05-08 15:39:18,1899.0,2017-05-08 15:42:00,1899,2017-05-08 15:04:12,00:37:48


In [43]:
def clean_otp_legs_actual_time_df(otp_legs_st_end_df):
    clean_legs_time = otp_legs_start_end[['date','user_trip_id','itinerary_id','leg_id','route','busCode','tripNum','from_stop_id','from_timestamp','to_stop_id','to_timestamp']] \
                .assign(actual_duration_mins = ((otp_legs_start_end['to_timestamp'] - otp_legs_start_end['from_timestamp'])/pd.Timedelta(minutes=1))) \
                .sort_values(by=['date','user_trip_id','itinerary_id','leg_id'])
            
    clean_legs_time = clean_legs_time[clean_legs_time['actual_duration_mins'] > 0]
    
    return clean_legs_time

clean_otp_legs_actual_time = clean_otp_legs_actual_time_df(otp_legs_start_end)

In [44]:
clean_otp_legs_actual_time.head()

,date,user_trip_id,itinerary_id,leg_id,route,busCode,tripNum,from_stop_id,from_timestamp,to_stop_id,to_timestamp,actual_duration_mins
0,2017-05-08,17179869196,1,2,901,MC305,7.0,29598.0,2017-05-08 11:12:05,33654.0,2017-05-08 11:38:21,26.266667
1,2017-05-08,17179869196,2,2,901,MC305,7.0,29598.0,2017-05-08 11:12:05,33654.0,2017-05-08 11:38:21,26.266667
2,2017-05-08,17179869196,3,2,901,MC303,5.0,29598.0,2017-05-08 11:24:52,33654.0,2017-05-08 11:47:43,22.850000
9,2017-05-08,94489280525,1,2,972,MC491,14.0,29599.0,2017-05-08 21:12:39,33375.0,2017-05-08 21:32:05,19.433333
16,2017-05-08,94489280525,2,2,965,MN606,3.0,26720.0,2017-05-08 21:33:33,33375.0,2017-05-08 21:50:12,16.650000


In [45]:
num_matched_bus_legs_st = len(clean_otp_legs_actual_time)
print "Num Bus Legs whose end was found:", num_matched_bus_legs_st, '(', 100*(num_matched_bus_legs_st/float(num_bus_legs)), '%)'


Num Bus Legs whose end was found: 125 ( 90.5797101449 %)


#### Cleaning Memory

In [46]:
#Clean Memory
#otp_legs_st.unpersist(blocking=True)
#bus_trips_data2.unpersist(blocking=True)
#clean_bus_trips_data2.unpersist(blocking=True)
#otp_legs_start_end.unpersist(blocking=True)


#### Enriching OTP suggestions legs with actual time data

In [47]:
def combine_otp_suggestions_with_bus_legs_actual_time(otp_suggestions,bus_legs_actual_time):
    matched_legs_suggestions = otp_suggestions \
                .merge(clean_otp_legs_actual_time, on=['date','user_trip_id','itinerary_id','leg_id', 'route', 'from_stop_id','to_stop_id'], how='left') \
                .assign(considered_duration_mins = lambda x: np.where(x['mode'] == 'BUS',
                                            x['actual_duration_mins'],
                                            x['otp_duration_mins']),
                        considered_start_time = lambda x: np.where(x['mode'] == 'BUS', 
                                         x['from_timestamp'], 
                                         x['otp_start_time']))
    return  matched_legs_suggestions

print "Enriching OTP suggestions legs with actual time data..."
all_legs_actual_time = combine_otp_suggestions_with_bus_legs_actual_time(otp_legs_df,clean_otp_legs_actual_time)


Enriching OTP suggestions legs with actual time data...


In [48]:
all_legs_actual_time.sort_values('user_trip_id').head(40)

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,busCode,tripNum,from_timestamp,to_timestamp,actual_duration_mins,considered_duration_mins,considered_start_time
0,2017-05-08,17179869196,1,1,2017-05-08 11:07:58,2017-05-08 11:07:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 11:07:58
1,2017-05-08,17179869196,1,2,2017-05-08 11:08:00,2017-05-08 11:31:20,BUS,901,29598.0,33654.0,23.333333,MC305,7.0,2017-05-08 11:12:05,2017-05-08 11:38:21,26.266667,26.266667,2017-05-08 11:12:05
2,2017-05-08,17179869196,1,3,2017-05-08 11:31:21,2017-05-08 11:31:42,WALK,NaN,NaN,NaN,0.350000,NaN,NaN,NaT,NaT,NaN,0.350000,2017-05-08 11:31:21
3,2017-05-08,17179869196,2,1,2017-05-08 11:16:58,2017-05-08 11:16:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 11:16:58
4,2017-05-08,17179869196,2,2,2017-05-08 11:17:00,2017-05-08 11:40:20,BUS,901,29598.0,33654.0,23.333333,MC305,7.0,2017-05-08 11:12:05,2017-05-08 11:38:21,26.266667,26.266667,2017-05-08 11:12:05
5,2017-05-08,17179869196,2,3,2017-05-08 11:40:21,2017-05-08 11:40:42,WALK,NaN,NaN,NaN,0.350000,NaN,NaN,NaT,NaT,NaN,0.350000,2017-05-08 11:40:21
6,2017-05-08,17179869196,3,1,2017-05-08 11:26:58,2017-05-08 11:26:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 11:26:58
7,2017-05-08,17179869196,3,2,2017-05-08 11:27:00,2017-05-08 11:51:16,BUS,901,29598.0,33654.0,24.266667,MC303,5.0,2017-05-08 11:24:52,2017-05-08 11:47:43,22.850000,22.850000,2017-05-08 11:24:52
8,2017-05-08,17179869196,3,3,2017-05-08 11:51:17,2017-05-08 11:51:38,WALK,NaN,NaN,NaN,0.350000,NaN,NaN,NaT,NaT,NaN,0.350000,2017-05-08 11:51:17
17,2017-05-08,94489280525,3,3,2017-05-08 22:19:03,2017-05-08 22:19:10,WALK,NaN,NaN,NaN,0.116667,NaN,NaN,NaT,NaT,NaN,0.116667,2017-05-08 22:19:03


#### Filtering out itineraries with bus legs not identified in bus data

In [49]:
def select_itineraries_fully_identified(otp_itineraries_legs):
    itineraries_not_fully_identified = otp_itineraries_legs[(otp_itineraries_legs['mode'] == 'BUS') & (pd.isnull(otp_itineraries_legs['busCode']))] \
                                        [['date','user_trip_id','itinerary_id']] \
                                        .drop_duplicates()
    itineraries_fully_identified = pd.concat([otp_itineraries_legs[['date','user_trip_id','itinerary_id']].drop_duplicates(),itineraries_not_fully_identified]) \
                                    .drop_duplicates(keep=False)
    return otp_itineraries_legs.merge(itineraries_fully_identified, on=['date','user_trip_id','itinerary_id'], how='inner')

print "Filtering out itineraries with bus legs not identified in bus data..."
clean_legs_actual_time = select_itineraries_fully_identified(all_legs_actual_time)

Filtering out itineraries with bus legs not identified in bus data...


In [50]:
clean_legs_actual_time

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,busCode,tripNum,from_timestamp,to_timestamp,actual_duration_mins,considered_duration_mins,considered_start_time
0,2017-05-08,17179869196,1,1,2017-05-08 11:07:58,2017-05-08 11:07:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 11:07:58
1,2017-05-08,17179869196,1,2,2017-05-08 11:08:00,2017-05-08 11:31:20,BUS,901,29598.0,33654.0,23.333333,MC305,7.0,2017-05-08 11:12:05,2017-05-08 11:38:21,26.266667,26.266667,2017-05-08 11:12:05
2,2017-05-08,17179869196,1,3,2017-05-08 11:31:21,2017-05-08 11:31:42,WALK,NaN,NaN,NaN,0.350000,NaN,NaN,NaT,NaT,NaN,0.350000,2017-05-08 11:31:21
3,2017-05-08,17179869196,2,1,2017-05-08 11:16:58,2017-05-08 11:16:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 11:16:58
4,2017-05-08,17179869196,2,2,2017-05-08 11:17:00,2017-05-08 11:40:20,BUS,901,29598.0,33654.0,23.333333,MC305,7.0,2017-05-08 11:12:05,2017-05-08 11:38:21,26.266667,26.266667,2017-05-08 11:12:05
5,2017-05-08,17179869196,2,3,2017-05-08 11:40:21,2017-05-08 11:40:42,WALK,NaN,NaN,NaN,0.350000,NaN,NaN,NaT,NaT,NaN,0.350000,2017-05-08 11:40:21
6,2017-05-08,17179869196,3,1,2017-05-08 11:26:58,2017-05-08 11:26:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 11:26:58
7,2017-05-08,17179869196,3,2,2017-05-08 11:27:00,2017-05-08 11:51:16,BUS,901,29598.0,33654.0,24.266667,MC303,5.0,2017-05-08 11:24:52,2017-05-08 11:47:43,22.850000,22.850000,2017-05-08 11:24:52
8,2017-05-08,17179869196,3,3,2017-05-08 11:51:17,2017-05-08 11:51:38,WALK,NaN,NaN,NaN,0.350000,NaN,NaN,NaT,NaT,NaN,0.350000,2017-05-08 11:51:17
9,2017-05-08,94489280525,1,1,2017-05-08 21:27:58,2017-05-08 21:27:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 21:27:58


In [51]:
len(clean_legs_actual_time)

379

In [52]:
def filter_bus_legs_matched_to_same_observed_itinerary(otp_matched_legs):
    return otp_matched_legs \
                .merge((clean_legs_actual_time[clean_legs_actual_time['mode'] == 'BUS'] \
                                .drop_duplicates(subset=['date','user_trip_id','from_stop_id','to_stop_id','busCode','otp_start_time']) \
                                [['date','user_trip_id','itinerary_id']].drop_duplicates()), \
                       on=['date','user_trip_id','itinerary_id'], \
                       how='inner')

print "Filtering out itineraries with matched bus legs duplicated for the same trip..."
clean_legs_actual_time = filter_bus_legs_matched_to_same_observed_itinerary(clean_legs_actual_time)

Filtering out itineraries with matched bus legs duplicated for the same trip...


In [53]:
len(clean_legs_actual_time)

367

In [54]:
def filter_itineraries_without_bus_legs(otp_itineraries_legs):
    itineraries_with_bus_legs = otp_itineraries_legs[otp_itineraries_legs['mode'] == 'BUS'] \
                                    [['date','user_trip_id','itinerary_id']] \
                                    .drop_duplicates()
    return otp_itineraries_legs.merge(itineraries_with_bus_legs, on=['date','user_trip_id','itinerary_id'], how='inner')


print "Filtering out itineraries without bus legs..."
clean_legs_actual_time = filter_itineraries_without_bus_legs(clean_legs_actual_time)

Filtering out itineraries without bus legs...


In [55]:
clean_legs_actual_time.head()

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,busCode,tripNum,from_timestamp,to_timestamp,actual_duration_mins,considered_duration_mins,considered_start_time
0,2017-05-08,17179869196,1,1,2017-05-08 11:07:58,2017-05-08 11:07:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 11:07:58
1,2017-05-08,17179869196,1,2,2017-05-08 11:08:00,2017-05-08 11:31:20,BUS,901,29598.0,33654.0,23.333333,MC305,7.0,2017-05-08 11:12:05,2017-05-08 11:38:21,26.266667,26.266667,2017-05-08 11:12:05
2,2017-05-08,17179869196,1,3,2017-05-08 11:31:21,2017-05-08 11:31:42,WALK,NaN,NaN,NaN,0.350000,NaN,NaN,NaT,NaT,NaN,0.350000,2017-05-08 11:31:21
3,2017-05-08,17179869196,2,1,2017-05-08 11:16:58,2017-05-08 11:16:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 11:16:58
4,2017-05-08,17179869196,2,2,2017-05-08 11:17:00,2017-05-08 11:40:20,BUS,901,29598.0,33654.0,23.333333,MC305,7.0,2017-05-08 11:12:05,2017-05-08 11:38:21,26.266667,26.266667,2017-05-08 11:12:05


In [56]:
clean_legs_actual_time[clean_legs_actual_time['mode'] == 'BUS'] \
        [['otp_start_time','otp_end_time','otp_duration_mins','from_timestamp','to_timestamp','actual_duration_mins']] \
        .head(10)

,otp_start_time,otp_end_time,otp_duration_mins,from_timestamp,to_timestamp,actual_duration_mins
1,2017-05-08 11:08:00,2017-05-08 11:31:20,23.333333,2017-05-08 11:12:05,2017-05-08 11:38:21,26.266667
4,2017-05-08 11:17:00,2017-05-08 11:40:20,23.333333,2017-05-08 11:12:05,2017-05-08 11:38:21,26.266667
7,2017-05-08 11:27:00,2017-05-08 11:51:16,24.266667,2017-05-08 11:24:52,2017-05-08 11:47:43,22.850000
10,2017-05-08 21:28:00,2017-05-08 21:50:13,22.216667,2017-05-08 21:12:39,2017-05-08 21:32:05,19.433333
13,2017-05-08 21:32:24,2017-05-08 21:59:39,27.250000,2017-05-08 21:33:33,2017-05-08 21:50:12,16.650000
16,2017-05-08 21:52:00,2017-05-08 22:19:02,27.033333,2017-05-08 21:52:11,2017-05-08 22:08:52,16.683333
19,2017-05-08 11:33:43,2017-05-08 11:38:56,5.216667,2017-05-08 11:42:54,2017-05-08 11:49:09,6.250000
22,2017-05-08 11:35:16,2017-05-08 11:40:53,5.616667,2017-05-08 12:17:47,2017-05-08 12:24:04,6.283333
25,2017-05-08 11:49:25,2017-05-08 11:54:55,5.500000,2017-05-08 12:17:47,2017-05-08 12:24:04,6.283333
28,2017-05-08 14:57:24,2017-05-08 15:09:51,12.450000,2017-05-08 15:06:06,2017-05-08 15:16:59,10.883333


#### Printing Fully Identified OTP Itineraries Stats

In [57]:
num_itineraries_fully_identified = len(clean_legs_actual_time[['user_trip_id','itinerary_id']].drop_duplicates())
print "Num Itineraries fully identified in BUSTE data:", num_itineraries_fully_identified, '(', 100*(num_itineraries_fully_identified/float(total_num_itineraries)), '%)'

Num Itineraries fully identified in BUSTE data: 123 ( 83.1081081081 %)


In [58]:
print "Writing OTP suggested itineraries legs with actual time to file..."
#clean_legs_actual_time.write.csv(path=results_folderpath+'/otp_legs_matched',header=True, mode='append')


Writing OTP suggested itineraries legs with actual time to file...


In [59]:
#Clean Memory
#clean_otp_legs_actual_time.unpersist(blocking=True)
#all_legs_actual_time.unpersist(blocking=True)

In [60]:
od_matrix.head()

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,o_unixtimestamp,next_o_unixtimestamp,leg_duration,dist,rn,user_trip_id,d_datetime,o_datetime,executed_duration,o_base_datetime
0,967,5,2287,6086229,-25.419019,-49.293322,4069.842,MN604,NaN,-25.418978,...,36114,40247,68.883333,0.003239,1,1451698946054,2017-05-08 10:13:34,2017-05-08 10:01:54,11.666667,2017-05-08 09:59:54
1,226,10,1799,6199865,-25.368517,-49.245174,5429.408,BA112,NaN,-25.368340,...,45060,42755,-1.000000,0.837961,1,558345748483,2017-05-08 13:05:01,2017-05-08 12:31:00,34.016667,2017-05-08 12:29:00
2,967,5,2286,6087798,-25.425386,-49.279591,1384.230,MN604,NaN,-25.425406,...,37511,45285,129.566667,0.000095,1,867583393796,2017-05-08 10:34:29,2017-05-08 10:25:11,9.300000,2017-05-08 10:23:11
3,821,1,2216,6061287,-25.399733,-49.333750,10405.611,MA008,NaN,-25.399771,...,22162,55281,551.983333,0.000000,1,816043786247,2017-05-08 06:27:31,2017-05-08 06:09:22,18.150000,2017-05-08 06:07:22
4,901,7,2257,6191372,-25.428454,-49.272983,8195.073,MC305,NaN,-25.428471,...,40078,52089,200.183333,0.570514,1,1056561954821,2017-05-08 11:12:05,2017-05-08 11:07:58,4.116667,2017-05-08 11:05:58


In [61]:
clean_legs_actual_time.dtypes

date                                object
user_trip_id                         int64
itinerary_id                         int64
leg_id                               int64
otp_start_time              datetime64[ns]
otp_end_time                datetime64[ns]
mode                                object
route                               object
from_stop_id                       float64
to_stop_id                         float64
otp_duration_mins                  float64
busCode                             object
tripNum                            float64
from_timestamp              datetime64[ns]
to_timestamp                datetime64[ns]
actual_duration_mins               float64
considered_duration_mins           float64
considered_start_time       datetime64[ns]
dtype: object

In [62]:
clean_legs_actual_time \
           .sort_values(['date','user_trip_id','itinerary_id','leg_id']) \
            .head(10)

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,busCode,tripNum,from_timestamp,to_timestamp,actual_duration_mins,considered_duration_mins,considered_start_time
0,2017-05-08,17179869196,1,1,2017-05-08 11:07:58,2017-05-08 11:07:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 11:07:58
1,2017-05-08,17179869196,1,2,2017-05-08 11:08:00,2017-05-08 11:31:20,BUS,901,29598.0,33654.0,23.333333,MC305,7.0,2017-05-08 11:12:05,2017-05-08 11:38:21,26.266667,26.266667,2017-05-08 11:12:05
2,2017-05-08,17179869196,1,3,2017-05-08 11:31:21,2017-05-08 11:31:42,WALK,NaN,NaN,NaN,0.350000,NaN,NaN,NaT,NaT,NaN,0.350000,2017-05-08 11:31:21
3,2017-05-08,17179869196,2,1,2017-05-08 11:16:58,2017-05-08 11:16:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 11:16:58
4,2017-05-08,17179869196,2,2,2017-05-08 11:17:00,2017-05-08 11:40:20,BUS,901,29598.0,33654.0,23.333333,MC305,7.0,2017-05-08 11:12:05,2017-05-08 11:38:21,26.266667,26.266667,2017-05-08 11:12:05
5,2017-05-08,17179869196,2,3,2017-05-08 11:40:21,2017-05-08 11:40:42,WALK,NaN,NaN,NaN,0.350000,NaN,NaN,NaT,NaT,NaN,0.350000,2017-05-08 11:40:21
6,2017-05-08,17179869196,3,1,2017-05-08 11:26:58,2017-05-08 11:26:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 11:26:58
7,2017-05-08,17179869196,3,2,2017-05-08 11:27:00,2017-05-08 11:51:16,BUS,901,29598.0,33654.0,24.266667,MC303,5.0,2017-05-08 11:24:52,2017-05-08 11:47:43,22.850000,22.850000,2017-05-08 11:24:52
8,2017-05-08,17179869196,3,3,2017-05-08 11:51:17,2017-05-08 11:51:38,WALK,NaN,NaN,NaN,0.350000,NaN,NaN,NaT,NaT,NaN,0.350000,2017-05-08 11:51:17
9,2017-05-08,94489280525,1,1,2017-05-08 21:27:58,2017-05-08 21:27:59,WALK,NaN,NaN,NaN,0.016667,NaN,NaN,NaT,NaT,NaN,0.016667,2017-05-08 21:27:58


#### Gather all trips alternative/executed itineraries info

In [63]:
first_boarding_time = clean_legs_actual_time[clean_legs_actual_time['mode'] == 'BUS'] \
                        .groupby(['date', 'user_trip_id', 'itinerary_id']) \
                        .agg({'otp_start_time':'first','considered_start_time':'first'}) \
                        .reset_index() \
                        .rename(index=str,columns={'otp_start_time': 'planned_start_time', 'considered_start_time': 'actual_start_time'}) \
                        .sort_values(['date','user_trip_id','itinerary_id'])        

first_boarding_time.head()

,date,user_trip_id,itinerary_id,actual_start_time,planned_start_time
0,2017-05-08,17179869196,1,2017-05-08 11:12:05,2017-05-08 11:08:00
1,2017-05-08,17179869196,2,2017-05-08 11:12:05,2017-05-08 11:17:00
2,2017-05-08,17179869196,3,2017-05-08 11:24:52,2017-05-08 11:27:00
3,2017-05-08,94489280525,1,2017-05-08 21:12:39,2017-05-08 21:28:00
4,2017-05-08,94489280525,2,2017-05-08 21:33:33,2017-05-08 21:32:24


In [64]:
user_trips_time_info = od_matrix.rename(index=str,columns={'executed_duration':'exec_duration_mins','o_datetime':'exec_start_time'}) \
                        [['date','user_trip_id','exec_duration_mins','exec_start_time']]

user_trips_time_info.head()

,date,user_trip_id,exec_duration_mins,exec_start_time
0,2017-05-08,1451698946054,11.666667,2017-05-08 10:01:54
1,2017-05-08,558345748483,34.016667,2017-05-08 12:31:00
2,2017-05-08,867583393796,9.300000,2017-05-08 10:25:11
3,2017-05-08,816043786247,18.150000,2017-05-08 06:09:22
4,2017-05-08,1056561954821,4.116667,2017-05-08 11:07:58


In [65]:
matched_executed_trips.head()

,date,user_trip_id,itinerary_id,planned_duration_mins,planned_start_time
0,2017-05-08,1451698946054,0,15.983333,2017-05-08 13:31:16
1,2017-05-08,867583393796,0,5.150000,2017-05-08 13:30:00
2,2017-05-08,816043786247,0,14.366667,2017-05-08 09:11:36
3,2017-05-08,996432412675,0,7.083333,2017-05-08 10:41:53
4,2017-05-08,1477468749834,0,20.750000,2017-05-08 10:08:30


In [66]:
executed_legs = user_trips_time_info \
            .merge(matched_executed_trips, on=['date','user_trip_id'], how='left') \
            .assign(actual_duration_mins = lambda x: x['exec_duration_mins'], \
                    actual_start_time = lambda x: x['exec_start_time'],
                    itinerary_id = 0) \
            [['date','user_trip_id','itinerary_id','planned_duration_mins','actual_duration_mins','exec_duration_mins',
                     'planned_start_time','actual_start_time','exec_start_time']]
            
executed_legs.head()

,date,user_trip_id,itinerary_id,planned_duration_mins,actual_duration_mins,exec_duration_mins,planned_start_time,actual_start_time,exec_start_time
0,2017-05-08,1451698946054,0,15.983333,11.666667,11.666667,2017-05-08 13:31:16,2017-05-08 10:01:54,2017-05-08 10:01:54
1,2017-05-08,558345748483,0,NaN,34.016667,34.016667,NaT,2017-05-08 12:31:00,2017-05-08 12:31:00
2,2017-05-08,867583393796,0,5.150000,9.300000,9.300000,2017-05-08 13:30:00,2017-05-08 10:25:11,2017-05-08 10:25:11
3,2017-05-08,816043786247,0,14.366667,18.150000,18.150000,2017-05-08 09:11:36,2017-05-08 06:09:22,2017-05-08 06:09:22
4,2017-05-08,1056561954821,0,NaN,4.116667,4.116667,NaT,2017-05-08 11:07:58,2017-05-08 11:07:58


In [67]:
matched_otp_legs = clean_legs_actual_time \
                            .groupby(['date', 'user_trip_id', 'itinerary_id']) \
                            .agg({'otp_duration_mins':'sum',
                                  'considered_duration_mins':'sum'}) \
                            .rename(index=str,columns={'otp_duration_mins':'planned_duration_mins',
                                                        'considered_duration_mins':'actual_duration_mins'}) \
                            .reset_index() \
                            .assign(user_trip_id = lambda x : pd.to_numeric(x['user_trip_id']), \
                                    itinerary_id = lambda x : pd.to_numeric(x['itinerary_id'])) \
                            .merge(first_boarding_time, on=['date','user_trip_id','itinerary_id'], how='inner') \
                            .merge(user_trips_time_info, on=['date','user_trip_id'], how='inner') \
                            .sort_values(['date','user_trip_id','itinerary_id']) \
                            [['date','user_trip_id','itinerary_id','planned_duration_mins','actual_duration_mins','exec_duration_mins',
                              'planned_start_time','actual_start_time','exec_start_time']]
                 
matched_otp_legs.head()

,date,user_trip_id,itinerary_id,planned_duration_mins,actual_duration_mins,exec_duration_mins,planned_start_time,actual_start_time,exec_start_time
0,2017-05-08,17179869196,1,23.700000,26.633333,16.016667,2017-05-08 11:08:00,2017-05-08 11:12:05,2017-05-08 07:59:32
1,2017-05-08,17179869196,2,23.700000,26.633333,16.016667,2017-05-08 11:17:00,2017-05-08 11:12:05,2017-05-08 07:59:32
2,2017-05-08,17179869196,3,24.633333,23.216667,16.016667,2017-05-08 11:27:00,2017-05-08 11:24:52,2017-05-08 07:59:32
3,2017-05-08,94489280525,1,22.350000,19.566667,19.833333,2017-05-08 21:28:00,2017-05-08 21:12:39,2017-05-08 18:04:31
4,2017-05-08,94489280525,2,32.300000,21.700000,19.833333,2017-05-08 21:32:24,2017-05-08 21:33:33,2017-05-08 18:04:31


In [68]:
executed_trips_with_sugestions_matched = matched_otp_legs[['user_trip_id']]\
                                            .drop_duplicates()
executed_trips_with_sugestions_matched.head()

,user_trip_id
0,17179869196
3,94489280525
6,163208757264
9,377957122060
12,386547056643


In [69]:
all_trips_alternatives = pd.concat([matched_otp_legs, executed_legs]) \
                .merge(executed_trips_with_sugestions_matched, on='user_trip_id',how='inner') \
                .sort_values(['date','user_trip_id','itinerary_id'])
        
all_trips_alternatives.head()

,date,user_trip_id,itinerary_id,planned_duration_mins,actual_duration_mins,exec_duration_mins,planned_start_time,actual_start_time,exec_start_time
3,2017-05-08,17179869196,0,23.333333,16.016667,16.016667,2017-05-08 11:08:00,2017-05-08 07:59:32,2017-05-08 07:59:32
0,2017-05-08,17179869196,1,23.700000,26.633333,16.016667,2017-05-08 11:08:00,2017-05-08 11:12:05,2017-05-08 07:59:32
1,2017-05-08,17179869196,2,23.700000,26.633333,16.016667,2017-05-08 11:17:00,2017-05-08 11:12:05,2017-05-08 07:59:32
2,2017-05-08,17179869196,3,24.633333,23.216667,16.016667,2017-05-08 11:27:00,2017-05-08 11:24:52,2017-05-08 07:59:32
7,2017-05-08,94489280525,0,22.216667,19.833333,19.833333,2017-05-08 21:28:00,2017-05-08 18:04:31,2017-05-08 18:04:31


In [70]:
all_trips_alternatives.to_csv(path_or_buf=results_folderpath + os.sep + 'part-00000',index=False)

In [71]:
print len(all_trips_alternatives), len(od_matrix), len(executed_trips_with_sugestions_matched)

171 50 48


In [72]:
print("--- Execution took %s seconds ---" % (time.time() - start_time))

--- Execution took 51.9312949181 seconds ---
